# 图像卷积

互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):                    #X为输入 ，K为核矩阵
    """计算二维互相关运算"""
    h,w  =K.shape                    #拿出核矩阵的高宽
    Y = torch.zeros((X.shape[0]-h+1 ,X.shape[1]-w+1))        #初始输出矩阵
    for i in range(Y.shape[0]):                       #遍历所有Y的元素，一一做计算
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()          #第i,j的输出 = X的i行到i+h行，j列到j+w列 （小区域）和K做点积，求和
    return Y

验证上述二维互相关运算的输出

In [4]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

## 1、实现二维卷积

In [ ]:
class Conv2d(nn.Module):
    def __init__(self,kernel_size):                           #kernel_size超参数
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self,x):
        return corr2d(x , self.weight)+self.bias

卷积层简单应用：检测图像中不同颜色的边缘

In [6]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [10]:
K = torch.tensor([[1.0,-1.0]])

输出Y中的1代表白色到黑色边缘，-1代表从黑色到白色的边缘

In [11]:
Y= corr2d(X ,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只能检测垂直边缘

In [13]:
corr2d(X.T,K)                 #X转置后就检测不出来了

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 2、给定X,Y,学习K

In [16]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)     #参数说明[输入通道，输出通道，核大小，偏执]

X= X.reshape((1,1,6,8))             #对于Convd来说输入都是4d的， 【输入通道数，批大小，高，宽】
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) **2          #均方误差
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad            #直接修改weight；裸写SGD
    if(i+1)%2 == 0:
        print(f'batch{i+1},loss{l.sum():.3f}')

batch2,loss6.114
batch4,loss1.028
batch6,loss0.173
batch8,loss0.029
batch10,loss0.005


所学卷积核的权重张量

In [17]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9888, -0.9855]])